# Georeferencing

This code uses the geographical information stored in the original unprocessed orthomosaic and applies it to a copy of the Seagrass binary tif, which is the end result of the previous section. The output is a binary tif that can be dragged and dropped into QGIS and has the correct geographical and projection information.

In [6]:
# import libraries
# gdal is short for Geospatial Data Abstraction Library and is also imbedded in QGIS (among others)
from osgeo import gdal

In [32]:
image_name = "AK_tryout"
src_path = rf"your\source\path\{image_name}.tif"
pro_path = rf"your\processed\path\{image_name}_SeagrassBinary.tif"
dst_path = rf"your\destination\path\{image_name}_GeoRef.tif"

In [38]:
def georeference(src_path, pro_path, dst_path):
    # open original and processed tif as datasets in gdal
    src_ds = gdal.Open(src_path)
    pro_ds = gdal.Open(pro_path)

    # check that there were no errors while loading the images
    if src_ds is None:
        print("The original tif has not loaded correctly.")
    if pro_ds is None:
        print("The binary tif has not loaded correctly.")

    # get geotransform data from the original tif and save as variable
    geo_data = src_ds.GetGeoTransform()
    
    # calculate the pixel area from the geographical data, necessary for part 6
    pixel_area = abs(geo_data[1]) * abs(geo_data[5]) * 10000

    # get driver
    fileformat = "GTiff"
    driver = gdal.GetDriverByName(fileformat)

    # create a copy of the processed image in the correct format (GTiff), then apply geotransform information to the copy
    dst_ds = driver.CreateCopy(dst_path, pro_ds, strict=0)
    dst_ds.SetGeoTransform(geo_data)

    # assign the right projection
    src_projection = src_ds.GetProjection()
    dst_ds.SetProjection(src_projection)
 
    if geo_data == dst_ds.GetGeoTransform():
        print(f"The binary file has been successfully georeferenced. The area of one pixel is {pixel_area} centimeters.")

    # close all datasets
    dst_ds = None
    src_ds = None
    pro_ds = None

In [39]:
georeference(src_path, pro_path, dst_path)

The binary file has been successfully georeferenced. The area of one pixel is 25.254630790658826 centimeters.
